In [ ]:

! kaggle datasets download -d mahmoudima/mma-facial-expression
! kaggle datasets download -d debanga/facial-expression-recognition-challenge

 97% 310M/321M [00:02<00:00, 115MB/s]
100% 321M/321M [00:02<00:00, 114MB/s]


In [ ]:
import zipfile
z=zipfile.ZipFile('/content/facial-expression-recognition-challenge.zip')
z1=zipfile.ZipFile('/content/mma-facial-expression.zip')
z.extractall()
z.close()
z1.extractall()
z1.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**firstly we are going to work with the csv data of the images writeen in the form of csv having pixels and labels of different emotions**

In [ ]:
import pandas as pd 
train=pd.read_csv('/content/icml_face_data.csv/icml_face_data.csv')
test=pd.read_csv('test.csv/test.csv')

In [ ]:
len(train)

35887

In [ ]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
train['emotion'] = train['emotion'].apply(lambda x: emotions[int(x)])
train.head()

,emotion,Usage,pixels
0,Angry,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,Angry,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,Fear,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,Sad,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,Neutral,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [ ]:
train['emotion'].value_counts()

Happy       8989
Neutral     6198
Sad         6077
Fear        5121
Angry       4953
Surprise    4002
Disgust      547
Name: emotion, dtype: int64

In [ ]:
emotion=['Happy','Neutral','Sad','Angry']
df=train[train.emotion.isin(emotion)]

In [ ]:
df.shape

(26217, 3)

In [ ]:
df

,emotion,Usage,pixels
0,Angry,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,Angry,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,Fear,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,Sad,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,Neutral,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...,...
35882,Neutral,PrivateTest,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883,Happy,PrivateTest,178 174 172 173 181 188 191 194 196 199 200 20...
35884,Angry,PrivateTest,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885,Happy,PrivateTest,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [ ]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

(26217, 4)

In [ ]:
le.classes_

array(['Angry', 'Happy', 'Neutral', 'Sad'], dtype=object)

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping # {3: 0, 4: 1, 6: 2}

{'Angry': 0, 'Happy': 1, 'Neutral': 2, 'Sad': 3}

In [ ]:
from keras.utils import np_utils
import numpy as np
img_array = df[' pixels'].apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)
img_array.shape

(26217, 48, 48, 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
  img_array,
  img_labels,
  shuffle=True, 
  stratify=img_labels,
  test_size=0.1, 
  random_state=42
)

In [ ]:
X_train = X_train / 255.
X_valid = X_valid / 255.

img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,MaxPool2D,Dense,Conv2D
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [ ]:
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation,MaxPooling2D,Conv2D
model_0=Sequential([
  Conv2D(64,3,activation='relu',input_shape=(img_height,img_width,img_depth)),
  MaxPooling2D(padding='same'),
  Conv2D(64,3,activation='relu'),
  MaxPooling2D(padding='same'),
  Conv2D(128,3,activation='relu'),
  MaxPooling2D(padding='same'), 
  Conv2D(128,3,activation='relu'),
  MaxPooling2D(padding='same'),  

  Flatten(),
  Dense(128,activation='relu'),
  Dense(num_classes,activation='softmax')
])


model_0.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07),
                metrics='accuracy')

model_0.fit(X_train,y_train,
            epochs=10,
            #steps_per_epoch=(1*len(X_train)),
            validation_data=(X_valid,y_valid))

Epoch 1/10
738/738 [==============================] - 16s 20ms/step - loss: 1.1823 - accuracy: 0.4645 - val_loss: 1.0625 - val_accuracy: 0.5236
Epoch 2/10
738/738 [==============================] - 14s 19ms/step - loss: 0.9552 - accuracy: 0.5915 - val_loss: 0.9350 - val_accuracy: 0.6049
Epoch 3/10
738/738 [==============================] - 14s 19ms/step - loss: 0.8606 - accuracy: 0.6375 - val_loss: 0.9570 - val_accuracy: 0.5953
Epoch 4/10
738/738 [==============================] - 14s 19ms/step - loss: 0.7939 - accuracy: 0.6761 - val_loss: 0.8745 - val_accuracy: 0.6411
Epoch 5/10
738/738 [==============================] - 14s 19ms/step - loss: 0.7305 - accuracy: 0.7064 - val_loss: 0.8491 - val_accuracy: 0.6568
Epoch 6/10
738/738 [==============================] - 14s 19ms/step - loss: 0.6669 - accuracy: 0.7356 - val_loss: 0.9001 - val_accuracy: 0.6362
Epoch 7/10
738/738 [==============================] - 14s 19ms/step - loss: 0.6026 - accuracy: 0.7629 - val_loss: 0.9127 - val_accuracy:

In [ ]:
model_0.evaluate(X_valid,y_valid)

82/82 [==============================] - 1s 8ms/step - loss: 1.0555 - accuracy: 0.6438


[1.0555319786071777, 0.6437833905220032]

## **now we will work with data of images stored in folders in the form of images**
1. FIRST METHOD WILL BE DOING THIS THE CONVENTIONAL WAY USINF CONVOLUTIONAL NEURAL NETWORK(model_1)
2.AFTER THAT WE WILL BE USING TRANSFER LEARNING AND FINE TUNING (model_2)

In [ ]:
import zipfile
z=zipfile.ZipFile('drive/MyDrive/train.zip')
z.extractall()
z.close()

In [ ]:
import zipfile
z=zipfile.ZipFile('drive/MyDrive/test.zip')
z.extractall()
z.close()

In [ ]:
train=image_dataset_from_directory('/content/train',
                                   label_mode='categorical',
                                   
                                   batch_size=32, image_size=(224, 224), shuffle=True, seed=42)
test=image_dataset_from_directory('/content/test',
                                   label_mode='categorical',
                                   batch_size=32, image_size=(224, 224), shuffle=True, seed=42)


Found 31878 files belonging to 5 classes.
Found 16010 files belonging to 5 classes.


In [ ]:
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation,MaxPooling2D,Conv2D
model_1=Sequential([
  Conv2D(32,3,activation='relu',input_shape=(224,224,3)),
  MaxPooling2D(padding='same'),
  Conv2D(64,3,activation='relu'),
  MaxPooling2D(padding='same'),
  Conv2D(64,3,activation='relu'),
  MaxPooling2D(padding='same'), 
  Conv2D(128,3,activation='relu'),
  MaxPooling2D(padding='same'),

  Flatten(),
  Dense(128,activation='relu'),
  Dense(5,activation='softmax')
])


model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics='accuracy')


model_1.fit(train,
            epochs=5,
            #steps_per_epoch=(1*len(X_train)),
            validation_data=(test),
            
            validation_steps=(0.15*len(test)))

In [ ]:
model_1.evaluate(test)

501/501 [==============================] - 21s 41ms/step - loss: 1.8090 - accuracy: 0.3505


[1.8089773654937744, 0.3504684567451477]

In [ ]:
model_1.save('emotion_csv.h5')

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
data_augmentation = Sequential([
  preprocessing.RandomFlip('horizontal'),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  preprocessing.RandomZoom(0.2),
  #preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNet                 
], name="data_augmentation")

In [ ]:
model=tf.keras.applications.EfficientNetB0(include_top=False)
model.trainable=False
input=tf.keras.layers.Input(shape=(224,224,3))
x=data_augmentation(input)
x=model(x,training=False)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(128,activation='relu')(x)
x=tf.keras.layers.Dense(64,activation='relu')(x)
outputs=tf.keras.layers.Dense(5,activation='softmax')(x)
model_1=tf.keras.Model(input,outputs)
model_1.summary()


16719872/16705208 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
data_augmentation (Sequentia (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
______________________________________________________________

In [ ]:
model_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics='accuracy')

history_data_aug=model_2.fit(train,
                                        epochs=5,
                                        steps_per_epoch=(len(train)),
                                        validation_data=(test),
                                        validation_steps=(0.15*len(test))
                                          )

Epoch 1/5
997/997 [==============================] - 331s 325ms/step - loss: 1.2809 - accuracy: 0.4752 - val_loss: 1.3365 - val_accuracy: 0.4531
Epoch 2/5
997/997 [==============================] - 233s 234ms/step - loss: 1.1647 - accuracy: 0.5319 - val_loss: 1.3799 - val_accuracy: 0.4420
Epoch 3/5
997/997 [==============================] - 207s 208ms/step - loss: 1.1311 - accuracy: 0.5491 - val_loss: 1.3219 - val_accuracy: 0.4667
Epoch 4/5
997/997 [==============================] - 185s 185ms/step - loss: 1.0913 - accuracy: 0.5623 - val_loss: 1.2940 - val_accuracy: 0.4774
Epoch 5/5
997/997 [==============================] - 176s 176ms/step - loss: 1.0629 - accuracy: 0.5800 - val_loss: 1.3495 - val_accuracy: 0.4544


In [ ]:
model.trainable = True

# Freeze all layers except for the
for layer in model.layers[:-10]:
  layer.trainable = False

# Recompile the model (always recompile after any adjustments to a model)
model_1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.0001), # lr is 10x lower than before for fine-tuning
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
initial_epochs=5
fine_tune_epochs = initial_epochs + 5

# Refit the model (same as model_2 except with more trainable layers)
history__data_aug = model_2.fit(train,
                                               epochs=fine_tune_epochs,
                                               initial_epoch=history_10_percent_data_aug.epoch[-1])

Epoch 5/10
997/997 [==============================] - 176s 170ms/step - loss: 1.0036 - accuracy: 0.6046
Epoch 6/10
997/997 [==============================] - 162s 162ms/step - loss: 0.9362 - accuracy: 0.6332
Epoch 7/10
997/997 [==============================] - 157s 157ms/step - loss: 0.8941 - accuracy: 0.6553
Epoch 8/10
997/997 [==============================] - 154s 154ms/step - loss: 0.8494 - accuracy: 0.6724
Epoch 9/10
997/997 [==============================] - 150s 150ms/step - loss: 0.8167 - accuracy: 0.6846
Epoch 10/10
997/997 [==============================] - 151s 151ms/step - loss: 0.7877 - accuracy: 0.6951


In [ ]:
model_2.save('emotions.h5')


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
